In [24]:
PREPROCESSED_SEQUENCE_PATH = '/home/bandeja/datasets/samsung-spatial/processed/2022-06-29-16-32-57_big_run_1_5fps/'

In [25]:
import cv2
import numpy as np
import os
from pathlib import Path

In [26]:
camera_folders = [name for name in os.listdir(PREPROCESSED_SEQUENCE_PATH) if os.path.isdir(os.path.join(PREPROCESSED_SEQUENCE_PATH, name))]
camera_folders

['4s', '1s', '2s', '3m']

### Create DVO configs

In [27]:
import shutil

for camera_folder in camera_folders:
    one_camera_data_path = os.path.join(PREPROCESSED_SEQUENCE_PATH, camera_folder)
    K_undist = np.load(os.path.join(one_camera_data_path, 'camera_params.npy'))

    new_config_path = os.path.join(one_camera_data_path, 'dvo.launch')
    shutil.copyfile('configs/dvo.launch', new_config_path)

    with open(new_config_path, 'r') as f:
        lines = f.readlines()
        lines[32] = '    <param type="double" name="fx" value="{0}" />\n'.format(K_undist[0, 0])
        lines[33] = '    <param type="double" name="fy" value="{0}" />\n'.format(K_undist[1, 1])
        lines[34] = '    <param type="double" name="cx" value="{0}" />\n'.format(K_undist[0, 2])
        lines[35] = '    <param type="double" name="cy" value="{0}" />\n'.format(K_undist[1, 2])

    with open(new_config_path, 'w') as f:
        f.writelines(lines)
    
    # DVO has hardcoded assocation file name (and also requires dense data)
    shutil.copyfile(os.path.join(one_camera_data_path, 'associations_dense.txt'), 
                    os.path.join(one_camera_data_path, 'assoc.txt'))